In [1]:
from pyspark.sql import SparkSession
import h5py
import os
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
import time
from pyspark.sql.functions import col, count
from pyspark.sql.functions import col, countDistinct

spark = SparkSession.builder \
    .master("spark://master:7077")\
    .appName("My PySpark Application") \
    .config("spark.cores.max", 4)\
    .getOrCreate()


start_time = time.time()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 22:00:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import col

#  path to file in HDFS
#hdfs_path = "hdfs://master:9000/user/ubuntu/data/duplicated_dataset.csv"
hdfs_path = "hdfs://master:9000/user/ubuntu/data/quadruple_dataset.csv"
#hdfs_path = "hdfs://master:9000/user/ubuntu/data/processedDataWithCountry.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(hdfs_path, header=True, inferSchema=True)

# Filter out rows with null or "Failed to retrieve" country values
filtered_df = df.filter((col("Country") != "NULL") & (col("Country") != "Failed to retrieve information"))

# Show the first few rows of the DataFrame
filtered_df.show()



+--------------------+--------------------+--------------------+--------------------+-------+
|              SongID|               Title|          ArtistName|         Artist_mbid|Country|
+--------------------+--------------------+--------------------+--------------------+-------+
|b'SOOIAPL12AB018C...|    b'Amsterdamaged'|          b'Humanzi'|838b705b-2389-4de...|     IE|
|b'SOBYHQA12AB0185...|"b""Do Somethin' ...|b'Mighty Mighty B...|779353f3-6401-4cd...|     US|
|b'SOTOMJM12A8C134...|    b'Milkcow Blues'|        b'Aerosmith'|3d2b98e5-556f-445...|     US|
|b'SOQATPT12AB0189...|        b'Mo lanmou'|b'Patrick Saint-E...|04559f8d-79ee-492...|     GP|
|b'SOPQXKA12AB0186...|b'Room At The Bot...|   b'Vince Guaraldi'|05b5488d-7141-4b2...|     US|
|b'SOOXNKG12A58A7C...|b'Someone Believe...|     b'Cindy Morgan'|84abfd2a-89a1-4eb...|     US|
|b'SOZMQND12A58A7D...|b'A Castle In The...|    b'Marty Robbins'|2c2add7c-9a99-481...|     US|
|b'SOSZDPD12A6D4F6...|b'Entre Autre Pas...|  b'Isabelle Adja

In [3]:
#What countries have the most artists? Top 10

# Group by 'Country' and count distinct artists
country_artists_count = filtered_df.groupBy('Country').agg(countDistinct('ArtistName').alias('ArtistCount')) \
    .orderBy(col('ArtistCount').desc())

# Show top 10 countries with the most artists
country_artists_count.show(10)

+-------+-----------+
|Country|ArtistCount|
+-------+-----------+
|     US|       1776|
|     GB|        374|
|     FR|        133|
|     DE|        111|
|     CA|         90|
|     JM|         86|
|     ES|         78|
|     BR|         66|
|     AR|         48|
|     SE|         47|
+-------+-----------+
only showing top 10 rows



In [4]:
#What artists have the most songs?  Top 10?

# Group by 'ArtistName' and count the number of songs
artist_song_count = filtered_df.groupBy('ArtistName') \
    .agg(count('SongID').alias('SongCount')) \
    .orderBy(col('SongCount').desc())

# Show top 10 artists with the most songs
artist_song_count.show(10)


+--------------------+---------+
|          ArtistName|SongCount|
+--------------------+---------+
|        b'Aerosmith'|       48|
|     b'Phil Collins'|       48|
|b'The Jackson Sou...|       48|
|     b'Sugar Minott'|       48|
|      b'Snow Patrol'|       44|
|        b'Radiohead'|       44|
|      b'Line Renaud'|       44|
|      b'Mikey Dread'|       44|
|b'Nick Cave & The...|       44|
|              b'Ayo'|       40|
+--------------------+---------+
only showing top 10 rows



In [5]:
#What countries have the most songs? Top 10

# Group by 'Country' and count the number of songs
country_song_count = filtered_df.groupBy('Country') \
    .agg(count('SongID').alias('SongCount')) \
    .orderBy(col('SongCount').desc())

# Show top 10 countries with the most songs
country_song_count.show(10)


+-------+---------+
|Country|SongCount|
+-------+---------+
|     US|    17012|
|     GB|     3820|
|     FR|     1388|
|     DE|     1128|
|     JM|      992|
|     CA|      784|
|     ES|      728|
|     BR|      616|
|     AR|      552|
|     FI|      384|
+-------+---------+
only showing top 10 rows



In [6]:
#Caluclate the time 
end_time = time.time()
elapsed_time = end_time - start_time

print("Time taken:", elapsed_time, "seconds")

Time taken: 23.147552490234375 seconds


In [7]:
spark.stop()